In [ ]:
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics.pairwise import cosine_similarity

In [13]:
import pandas as pd
df = pd.read_csv('developer_dataset.csv')

In [14]:
df.head()


,user_id,tech_field,programming_languages,region,availability,goals,experience
0,1,IoT,"Java, Arduino",Chandigarh,Part-time availability,Connect devices to the internet,10
1,2,Cybersecurity,"Python, C, Metasploit, Wireshark, Nmap",Mumbai,Full-time availability,"Protect systems from cyber threats, Enhance ne...",4
2,3,Game Development,"C++, Unity, Unreal Engine, C#, JavaScript",Ahmedabad,Available for projects,Work on 2D and 3D game development,10
3,4,Web3,"JavaScript, WebAssembly, Rust, Go",Jaipur,Part-time availability,"Create decentralized applications (DApps), Imp...",9
4,5,UI/UX Design,"Adobe XD, Sketch, Figma, InVision",Kolkata,Full-time availability,Create appealing user interfaces,3


In [15]:
import string

# Preprocess text data
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()

    # Remove punctuation
    text = ''.join([char for char in text if char not in string.punctuation])

    # Tokenize the text (split into words)
    tokens = text.split()

    return ' '.join(tokens)

# Apply preprocessing to relevant columns
df['tech_field'] = df['tech_field'].apply(preprocess_text)
df['programming_languages'] = df['programming_languages'].apply(preprocess_text)
df['goals'] = df['goals'].apply(preprocess_text)


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the text columns
tfidf_matrix = tfidf_vectorizer.fit_transform(df['tech_field'] + ' ' + df['programming_languages'] + ' ' + df['goals'])

# Check the shape of the TF-IDF matrix
print(tfidf_matrix.shape)


(500, 127)


In [20]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate cosine similarity between users
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Check the shape of the cosine similarity matrix
print(cosine_sim.shape)

# Example: Get recommendations for user with user_id = 1
user_id = 1
sim_scores = list(enumerate(cosine_sim[user_id - 1]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

(500, 500)


In [26]:
def get_recommendations(user_id, cosine_sim=cosine_sim):
    # Get the index of the user
    user_index = df[df['user_id'] == user_id].index[0]

    # Get the pairwise cosine similarity scores
    sim_scores = list(enumerate(cosine_sim[user_index]))

    # Sort the developers by their similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the top recommendations (excluding the user's own profile)
    top_recommendations = sim_scores[1:6]  # Change the number of recommendations as needed

    # Get the user IDs of recommended developers
    recommended_user_ids = [df['user_id'][i] for i, _ in top_recommendations]

    return recommended_user_ids

# Input system to get the user's ID
user_id = int(input("Enter your user ID: "))

# Get recommendations for the entered user ID
recommendations = get_recommendations(user_id)

# Print the user's profile
print("User Profile:")
user_profile = df[df['user_id'] == user_id]
print(user_profile)

# Print recommendations
print("Recommendations for user", user_id, ":")
for recommended_user_id in recommendations:
    recommended_profile = df[df['user_id'] == recommended_user_id]
    print("User ID:", recommended_user_id)
    print(recommended_profile)

Enter your user ID: 8
User Profile:
   user_id  tech_field                   programming_languages     region  \
7        8  mobile app  dart, objective-c, kotlin, java, swift  Hyderabad   

             availability                                              goals  \
7  Available for projects  build user-friendly mobile apps, explore cross...   

   experience                                      combined_text  
7           6  mobile app dart, objective-c, kotlin, java, sw...  
Recommendations for user 8 :
User ID: 9
   user_id  tech_field                   programming_languages  region  \
8        9  mobile app  objective-c, kotlin, java, dart, swift  Jaipur   

             availability                                              goals  \
8  Available for projects  build user-friendly mobile apps, explore cross...   

   experience                                      combined_text  
8           6  mobile app objective-c, kotlin, java, dart, sw...  
User ID: 61
    user_id  tech_

In [27]:
from prettytable import PrettyTable

# Define a function to format and print the recommendations
def print_recommendations(user_id, recommendations):
    user_profile = df[df['user_id'] == user_id]

    # Create a table to display user's profile
    user_table = PrettyTable()
    user_table.field_names = user_profile.columns
    user_table.add_row(user_profile.values[0])

    # Create a table to display recommendations
    recommendations_table = PrettyTable()
    recommendations_table.field_names = user_profile.columns

    for recommended_user_id in recommendations:
        recommended_profile = df[df['user_id'] == recommended_user_id]
        recommendations_table.add_row(recommended_profile.values[0])

    # Print user's profile and recommendations
    print("User Profile:")
    print(user_table)
    print("\nRecommendations for user", user_id, ":")
    print(recommendations_table)

# Input system to get the user's ID
user_id = int(input("Enter your user ID: "))

# Get recommendations for the entered user ID
recommendations = get_recommendations(user_id)

# Print the user's profile and recommendations
print_recommendations(user_id, recommendations)

Enter your user ID: 5
User Profile:
+---------+--------------+-----------------------------------+---------+------------------------+----------------------------------+------------+---------------------------------------------------------------------------------+
| user_id |  tech_field  |       programming_languages       |  region |      availability      |              goals               | experience |                                  combined_text                                  |
+---------+--------------+-----------------------------------+---------+------------------------+----------------------------------+------------+---------------------------------------------------------------------------------+
|    5    | ui/ux design | adobe xd, sketch, figma, invision | Kolkata | Full-time availability | create appealing user interfaces |     3      | ui/ux design adobe xd, sketch, figma, invision create appealing user interfaces |
+---------+--------------+--------------------------